# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.oncvpsp3.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397864285737                   -0.90    5.2   26.9ms
  2   -8.400246182102       -2.62       -1.74    1.0   18.8ms
  3   -8.400405772974       -3.80       -2.96    1.5   19.9ms
  4   -8.400428699053       -4.64       -2.97    3.2   47.3ms
  5   -8.400428806124       -6.97       -3.04    1.0   19.2ms
  6   -8.400429017971       -6.67       -4.50    1.0   19.1ms
  7   -8.400429023938       -8.22       -4.43    2.2   23.5ms
  8   -8.400429024370       -9.37       -4.93    1.0   19.7ms
  9   -8.400429024418      -10.32       -6.59    1.2   35.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397720295861                   -0.90           5.0    507ms
  2   -8.400383326040       -2.57       -1.78   0.80    2.2    568ms
  3   -8.400423103183       -4.40       -3.01   0.80    1.0    183ms
  4   -8.400428968312       -5.23       -3.41   0.80    2.5   64.4ms
  5   -8.400429021212       -7.28       -4.46   0.80    1.2   17.0ms
  6   -8.400429024361       -8.50       -5.54   0.80    2.5   20.4ms
  7   -8.400429024420      -10.23       -6.17   0.80    2.5   21.1ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.931825400248                   -1.04    2.98s
  2   -1.682723785687        0.42       -0.64    113ms
  3   -3.676851491297        0.30       -0.44   39.8ms
  4   -4.550129988430       -0.06       -0.51   64.3ms
  5   -6.414312389356        0.27       -0.57   39.4ms
  6   -7.219087678379       -0.09       -0.85   39.1ms
  7   -7.301743285670       -1.08       -1.37   28.8ms
  8   -7.798915034207       -0.30       -1.41   45.0ms
  9   -8.085139356837       -0.54       -1.11   39.8ms
 10   -8.201585371095       -0.93       -1.39   39.6ms
 11   -8.251322607592       -1.30       -1.53   39.1ms
 12   -8.253401346874       -2.68       -1.65   43.0ms
 13   -8.273405939566       -1.70       -1.82   39.7ms
 14   -8.317569012213       -1.35       -1.85   53.1ms
 15   -8.340647708667       -1.64       -1.95   29.0ms
 16   -8.379879939397       -1.41       -2.53   32.0ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397775338969                   -0.90    5.0   28.0ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400428849238                   -1.78    25.3s
  2   -8.400429024420       -6.76       -4.03    2.14s
  3   -8.400429024420      -14.45       -7.83   98.3ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.770657396281784e-7
|ρ_newton - ρ_scfv| = 1.2172839874541666e-7
|ρ_newton - ρ_dm|   = 1.0646301724645359e-6
